# Networks Assignment 

---

> Nikolaos Antonopoulos <br />
> Department of Management Science and Technology <br />
> Athens University of Economics and Business <br />
> antonopoulosn@hotmail.com

### Importing some packages

In [23]:
import pandas as pd
import os
import xlsxwriter

* We want to get the countries with the most imports for the year 2019.  
* Going to [Wikipedia](https://en.wikipedia.org/wiki/List_of_countries_by_imports), we notice that the recent data are for 2020.  
* Luckily, through wikipedia you can visit the older version of sites.  
* So, we go to the appropriate version of [Wikipedia](https://en.wikipedia.org/w/index.php?title=List_of_countries_by_imports&oldid=987487172) and we use <u> scraping</u> to get the top countries. 

In [24]:
import requests
from bs4 import BeautifulSoup
r = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_countries_by_imports&oldid=987487172")
html = r.text
soup = BeautifulSoup(html, 'html.parser')

* We get the countries, ascended by imports amount

In [25]:
flag = False
world_countires = []
review_titles_list = soup.find_all("a")
for review_title in review_titles_list:
    if (flag==True):
        if all(x.isalpha() or x.isspace() for x in review_title.get_text()):
            world_countires.append(review_title.get_text())
    if (review_title.get_text()=='[1]'):
        flag = True
    if (len(world_countires) >= 180):
        break


* The list, now looks like this

In [26]:
world_countires[:6]

['United States', 'European Union', 'China', 'Germany', 'Japan', 'France']

* Creating an array of the directory with the xlsx files

In [28]:
list_of_excel_sheets=[]
for filename in os.listdir("Excel_countries/top100"):
        list_of_excel_sheets.append(filename)

* Creating an array with the countries names without the xlsx.

In [29]:
excel_names_without_xlsx = list_of_excel_sheets.copy()
counter = 0
for i in excel_names_without_xlsx:
    excel_names_without_xlsx[counter] = i[:-5]
    counter = counter + 1

* We replace some country names, with the official ones

We also need to rename some countries, with their official name.  
This action is needed so they can match with our csv files.

In [30]:
dic = {'Hong Kong':'Hong Kong, China', 'Russia':'Russian Federation', 'Slovakia':'Slovak Republic','Egypt':'Egypt, Arab Rep.','South Korea':'Korea, Rep.',
'Serbia':'Serbia, FR(SerbiaMontenegro)','Macau':'Macao'}
world_countires = [dic.get(n, n) for n in world_countires]

#### Let's now do find the top 10 importers for each country

Assigning id for each country

In [31]:
def my_function(x):
    if x in excel_names_without_xlsx:
        return True
    else:
        return False
world_countires = [x for x in world_countires if my_function(x)]

* How many countries do i want

In [32]:
world_countires = world_countires[:80]

In [33]:
country_id  = pd.DataFrame(data=world_countires)
country_id.reset_index(inplace=True)
country_id.rename(columns = {0:'Label','index':'Id'}, inplace = True)

In [34]:
column_names = ['Source','Target','Type','Weight']

In [37]:
data = pd.DataFrame(columns = column_names)
for i in list_of_excel_sheets:
    counterrere = 0 
    if(i[:-5] in world_countires):
        # print(i)
        counter_of_countries_inserted = 0
        sheets = pd.read_excel('Excel_countries/top100/'+i,sheet_name='Product-TimeSeries-Product') 
        sheets = sheets.sort_values(by=['2019'], ascending=False)
        sheets['2019'] =sheets['2019'].round()
        for index, row in sheets.iterrows():
            if(row['Reporter Name'] in world_countires):
                counter_of_countries_inserted = counter_of_countries_inserted + 1
                # print(row['Partner Name'], row['Reporter Name'])
                df2 = pd.DataFrame({'Target':[country_id['Id'].loc[country_id['Label']==row['Partner Name']].iloc[0]],
                        'Source' : [country_id['Id'].loc[country_id['Label']==row['Reporter Name']].iloc[0]] ,
                        'Type' : ['Directed'],
                        'Weight': [row['2019']]})
                data = data.append(df2)
                if (counter_of_countries_inserted == 16):
                    break
data

,Source,Target,Type,Weight
0,27,48,Directed,10552232.0
0,1,48,Directed,7392262.0
0,0,48,Directed,5120074.0
0,43,48,Directed,3537027.0
0,17,48,Directed,3249418.0
...,...,...,...,...
0,5,17,Directed,6017699.0
0,12,17,Directed,5266632.0
0,20,17,Directed,5010304.0
0,22,17,Directed,4726448.0


* Finding European countires

In [38]:
European = [
'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Ireland',
'Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal','Romania','Slovak Republic','Slovenia','Spain','Sweden']

In [39]:
list_if_european = []
for index, row in country_id.iterrows():
    if(row.Label in European):
        list_if_european.append('Yes')
    else:
        list_if_european.append('No')
list_if_european
country_id['isEuropean'] = list_if_european

* For later on, it would also be useful to have the continents of each country.  
* pycountry library helps us with that. 

In [40]:
import pycountry_convert as pc
continents_of_world_countires = []
did_not_find_continent = []
for i in world_countires:
    try:
        country_code = pc.country_name_to_country_alpha2(i, cn_name_format="default")
        continent_name = pc.country_alpha2_to_continent_code(country_code)
        continents_of_world_countires.append(continent_name)
    except:
        did_not_find_continent.append(i)
        continents_of_world_countires.append(i)

* We notice that the library couldnt find 4 of our countries continents, so we map them by hand

In [41]:
dic = {'Hong Kong, China':'AS',
'Korea, Rep.':'AS',
'Egypt, Arab Rep.':'AF',
'Serbia, FR(SerbiaMontenegro)':'EU'}
continents_of_world_countires = [dic.get(n, n) for n in continents_of_world_countires]

In [42]:
country_id['continent'] = continents_of_world_countires

In [43]:
from collections import Counter
Counter(continents_of_world_countires)

Counter({'NA': 6, 'AS': 26, 'EU': 32, 'OC': 2, 'SA': 7, 'AF': 7})

In [44]:
data.to_excel("output.xlsx", index=False) 
writer = pd.ExcelWriter('country_id.xlsx', engine='xlsxwriter')
country_id.to_excel(writer, sheet_name='country', index=False)
writer.save()